Загрузка библиотек

In [8]:
import os
import shutil
import json
import cv2
import numpy as np
from google.colab import drive
from google.colab.patches import cv2_imshow
from pycocotools.coco import COCO


Добавление гугл диска для хранения датасетов и другой необходимой информации

In [ ]:
def mount_drive():
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
        print("Google Drive смонтирован")
    else:
        print("Google Drive уже подключен")

mount_drive()

Директории для хранения датасета, модели и папок с изображениями для обработки

In [ ]:
MAIN = '/content/drive/MyDrive/YOLO_COCO'
DATASET = os.path.join(MAIN, 'dataset')
ANNOTATIONS = os.path.join(DATASET, 'annotations')
MODEL = os.path.join(MAIN, 'model')
INPUT = '/content/input'
RESULT = '/content/result'
CROPPED = '/content/cropped'
os.makedirs(ANNOTATIONS, exist_ok=True)
os.makedirs(MODEL, exist_ok=True)
os.makedirs(INPUT, exist_ok=True)
os.makedirs(RESULT, exist_ok=True)
os.makedirs(CROPPED, exist_ok=True)

Проверяем есть ли датасет в директории, если нет начинаем загрузку

In [ ]:
def load_dataset():
    required_files = {
        'train': 'instances_train2017.json',
        'val': 'instances_val2017.json'
    }
    if not all(os.path.exists(os.path.join(ANNOTATIONS, f)) for f in required_files.values()):
        print("Загрузка датасета")
        !wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P {DATASET}
        !unzip -qo {DATASET}/annotations_trainval2017.zip -d {DATASET}
        !rm {DATASET}/annotations_trainval2017.zip

Целевые классы (изменить) и работа с датасетом

In [ ]:
CLASS_NAMES = ['aeroplane', 'bird', 'kite']

def filter_coco(annotation_path, output):
    coco = COCO(annotation_path)
    class_ids = coco.getCatIds(catNms=CLASS_NAMES)
    img_ids = coco.getImgIds(catIds=class_ids)

    new_ann = {
        'images': coco.loadImgs(img_ids),
        'annotations': coco.loadAnns(coco.getAnnIds(imgIds=img_ids, catIds=class_ids)),
        'categories': [c for c in coco.dataset['categories'] if c['name'] in CLASS_NAMES]
    }

    with open(os.path.join(output, 'filtered_annotations.json'), 'w') as f:
        json.dump(new_ann, f, indent=2)

Установка фреймворка даркнет для работы с Yolo, установка зависимостей cuda
Добавляем в makefile поддержку gpu opencv и cudnn

In [ ]:
def load_yolo():
    if not os.path.exists('darknet'):
        !git clone https://github.com/AlexeyAB/darknet
    %cd darknet
    !sed -i 's/OPENCV=0/OPENCV=1/' Makefile
    !sed -i 's/GPU=0/GPU=1/' Makefile
    !sed -i 's/CUDNN=0/CUDNN=1/' Makefile
    !make
    %cd ..
    !apt-get install -y --no-install-recommends cuda-toolkit-11-2
    !cp darknet/cfg/yolov3-tiny.cfg {MODEL}/yolov3-tiny-custom.cfg
    !sed -i 's/classes=80/classes=3/g' {MODEL}/yolov3-tiny-custom.cfg
    !sed -i 's/filters=255/filters=24/g' {MODEL}/yolov3-tiny-custom.cfg
    with open(f"{MODEL}/obj.names", 'w') as f:
        f.write('\n'.join(CLASS_NAMES))

Детекция обьектов предобучнной моделью
1. Загрузка готовых весов yolo обученных на coco
2. Находим указанные классы
3. Загружаем модель 
4. Очищаем результаты предыдущих детекций
5. Преобразуем изображение и для каждого предсказанного класса смотрим уверенность
5.1 Удаляем повторные ббоксы на одном обьекте с помощью NMS
6. Сортируем в зависимости от уверенности

In [ ]:
def detection():
    if not os.path.exists('yolov3-tiny.weights'):
        !wget https://pjreddie.com/media/files/yolov3-tiny.weights
    with open("darknet/data/coco.names", "r") as f:
        coco_classes = [line.strip() for line in f.readlines()]

    class_ids = []
    for cls in CLASS_NAMES:
        if cls in coco_classes:
            class_ids.append(coco_classes.index(cls))
        else:
            print(f"'{cls}' нет в coco")

    net = cv2.dnn.readNetFromDarknet('darknet/cfg/yolov3-tiny.cfg', 'yolov3-tiny.weights')
    ln = net.getLayerNames()
    ln = [ln[i-1] for i in net.getUnconnectedOutLayers()]

    shutil.rmtree(RESULT, ignore_errors=True)
    shutil.rmtree(CROPPED, ignore_errors=True)
    os.makedirs(RESULT, exist_ok=True)
    os.makedirs(CROPPED, exist_ok=True)

    for filename in os.listdir(INPUT):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(INPUT, filename)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Проблема с файлом {filename}")
                continue
            H, W = img.shape[:2]
            result = img.copy()
            high_confidence = False
            blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
            net.setInput(blob)
            outputs = net.forward(ln)
            boxes = []
            confidences = []
            class_ids_detected = []
            for i in outputs:
                for detection in i:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.3 and class_id in class_ids: 
                        box = detection[0:4] * np.array([W, H, W, H])
                        (centerX, centerY, width, height) = box.astype("int")
                        x = max(0, int(centerX - (width / 2)))
                        y = max(0, int(centerY - (height / 2)))

                        boxes.append([x, y, int(width), int(height)])
                        confidences.append(float(confidence))
                        class_ids_detected.append(class_id)

            indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
            print(f"Изображение: {filename}")
            print(f"Найдено объектов: {len(indices)}")
            print(f"Предварительный класс: {class_id}")
            if len(indices) > 0:
                for i in indices.flatten():
                    x, y, w, h = boxes[i]
                    confidence = confidences[i]
                    class_name = coco_classes[class_ids_detected[i]]

                    if confidence >= 0.7:
                        high_confidence = True
                        label = f"{class_name} {confidence:.2f}"
                        color = (0, 255, 0)
                        cv2.rectangle(result, (x, y), (x+w, y+h), color, 2)
                        cv2.putText(result, label, (x, y-5),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    else:
                        crop = img[y:y+h+30, x:x+w+30]
                        crop_filename = f"{os.path.splitext(filename)[0]}_{i}.jpg"
                        cv2.imwrite(os.path.join(CROPPED, crop_filename), crop)
                if high_confidence:
                    output_path = os.path.join(RESULT, filename)
                    cv2.imwrite(output_path, result)
            else:
                print("Пустое изображение")


Код для будущего обучения елки
Три класса, train.txt список изобдажений для обучения (должны быть указаны пути), val для валидации,names имена классов, backup для сохранения весов во время обучения
obj.names создаваться должен по идее при настройке yolo выше

In [ ]:
def train_yolo():
    config = f"""classes = 3
    train = {DATASET}/train.txt
    valid = {DATASET}/val.txt
    names = {MODEL}/obj.names
    backup = {MODEL}/backup"""
    with open(f"{MODEL}/obj.data", 'w') as f:
        f.write(config)
    !./darknet/darknet detector train \
        {MODEL}/obj.data \
        {MODEL}/yolov3-tiny-custom.cfg \
        -dont_show \
        -map \
        -clear

Запуск детекции

In [ ]:
load_dataset()
filter_coco(os.path.join(ANNOTATIONS, 'instances_train2017.json'), DATASET)
load_yolo()
detection()
options = input("Начать обучение(ответить n): ")
if options.lower() == 'y':
    train_yolo()
else:
    print("done")